# Teacher (ChatGPT) → Dataset → Student (Ollama/Qwen) + Versionado + Knowledge Base Log

Este notebook implementa un flujo **mínimo y práctico**:

1. Leer un **PDF** (factura) y extraer **texto plano**
2. Llamar a **OpenAI (ChatGPT) como *teacher*** para etiquetar/estructurar la salida en **JSON estricto** (validado con **Pydantic**)
3. Preparar una muestra en **JSONL** para entrenar un **SLM en local** (p. ej. Qwen/Llama con LoRA usando una tool de fine-tuning externa)
4. Hacer una llamada básica a **Ollama/Qwen** para inferencia (como *student*) usando JSON mode
5. Guardar **versiones** (artifacts) y registrar un **log tipo knowledge base** (JSONL + SQLite opcional)

> Generado: 2026-01-05


## 0) Instalación (si hace falta)
Ejecuta esta celda si estás en un entorno limpio.

- `pdfplumber`: extrae texto de PDFs (si el PDF es escaneado, necesitarás OCR)
- `openai`: SDK OpenAI
- `pydantic`: validación de JSON
- `requests`: llamadas a Ollama


In [1]:
# !pip -q install pdfplumber openai pydantic requests python-dotenv

import sys, platform
print("Python:", sys.version)
print("Platform:", platform.platform())


Python: 3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 16:37:03) [MSC v.1929 64 bit (AMD64)]
Platform: Windows-11-10.0.26200-SP0


## 1) Configuración
Configura:
- Ruta del PDF
- OpenAI API Key (por env var)
- Modelo teacher
- Modelo student en Ollama (ej: `qwen2.5:7b-instruct`)
- Directorios de salida para versionado y knowledge base


In [ ]:
from pathlib import Path
import os, json, hashlib, datetime
import pdfplumber
import requests
from pydantic import BaseModel, Field
from typing import List, Optional

# --- INPUTS ---
PDF_PATH = Path("facturas_compras_sample01.pdf")  # cambia si lo necesitas

# OpenAI
OPENAI_MODEL = "gpt-4o-mini"  # teacher: coste bajo + structured outputs
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")  # export OPENAI_API_KEY=...
# Ollama
OLLAMA_URL = os.getenv("OLLAMA_URL", "http://localhost:11434")
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "qwen2.5:7b-instruct")  # asegúrate de tenerlo en ollama

# --- OUTPUTS ---
RUN_ID = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
ARTIFACTS_DIR = Path("artifacts") / RUN_ID
KB_DIR = Path("knowledge_base")
ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)
KB_DIR.mkdir(parents=True, exist_ok=True)

print("ARTIFACTS_DIR:", ARTIFACTS_DIR.resolve())
print("KB_DIR:", KB_DIR.resolve())
print("PDF_PATH exists?", PDF_PATH.exists())


## 2) Extraer texto plano del PDF
Primero intentamos extraer texto con `pdfplumber`.

⚠️ Si el PDF es un escaneo (imagen), `pdfplumber` puede devolver poco texto. En ese caso, usa OCR (Textract/Tesseract).


In [ ]:
def extract_text_pdfplumber(pdf_path: Path) -> str:
    texts = []
    with pdfplumber.open(str(pdf_path)) as pdf:
        for page in pdf.pages:
            t = page.extract_text() or ""
            texts.append(t)
    return "\n\n".join(texts).strip()

raw_text = extract_text_pdfplumber(PDF_PATH)
print("Chars:", len(raw_text))
print(raw_text[:800])


## 3) Schema Pydantic (salida del teacher y del student)
Estructura fija, parseable, ideal para auditoría y entrenamiento.


In [ ]:
class SensitivityLabel(BaseModel):
    label: int = Field(..., description="0=no sensible, 1=sensible")
    confidence: float = Field(..., ge=0.0, le=1.0)
    signals: List[str] = Field(default_factory=list)
    rationale_short: str

    @classmethod
    def from_any(cls, obj: object) -> "SensitivityLabel":
        # helper para validar desde dict o str JSON
        if isinstance(obj, str):
            return cls.model_validate_json(obj)
        return cls.model_validate(obj)


## 4) Prompt del teacher (ChatGPT)
Forzamos JSON estricto (sin texto adicional).


In [ ]:
TEACHER_SYSTEM = (
    "Eres un analista de protección de datos. "
    "Clasifica documentos administrativos/financieros por sensibilidad."
)

TEACHER_INSTRUCTIONS = """Devuelve SOLO un JSON válido con esta estructura EXACTA:
{
  "label": 0|1,
  "confidence": 0.0-1.0,
  "signals": ["..."],
  "rationale_short": "una frase corta"
}

Criterio:
- label=1 (SENSIBLE) si aparece IBAN, NIF/CIF, datos bancarios, datos de pago, identificadores fiscales o información financiera identificable.
- label=0 si no hay PII/finanzas identificables.
- rationale_short: 1 frase, sin pasos intermedios.
"""

def build_teacher_user_prompt(doc_text: str) -> str:
    return f"""Texto del documento:
<<<
{doc_text}
>>>
"""


## 5) Llamada a OpenAI (teacher) con Structured Outputs + parse Pydantic
Este patrón sigue la guía oficial: schema basado en Pydantic.
Si tu SDK no tiene `responses.parse`, usa `response_format=json_schema` manual.


In [ ]:
from openai import OpenAI

def sha256_text(s: str) -> str:
    return hashlib.sha256(s.encode("utf-8")).hexdigest()

DOC_HASH = sha256_text(raw_text)
print("DOC_HASH:", DOC_HASH)

def teacher_label_document(doc_text: str) -> SensitivityLabel:
    if not OPENAI_API_KEY:
        raise RuntimeError("OPENAI_API_KEY no está configurada en el entorno.")

    client = OpenAI(api_key=OPENAI_API_KEY)

    resp = client.responses.parse(
        model=OPENAI_MODEL,
        instructions=TEACHER_SYSTEM + "\n\n" + TEACHER_INSTRUCTIONS,
        input=build_teacher_user_prompt(doc_text),
        text_format=SensitivityLabel,
    )
    return resp.output_parsed

# --- Ejecutar teacher ---
# teacher_out = teacher_label_document(raw_text)
# print(teacher_out.model_dump())

print("Descomenta las líneas de arriba para ejecutar la llamada real al teacher.")


## 6) Guardar artefactos del teacher (versionado)
Guardamos texto, label y metadata (modelo/prompt/timestamps).


In [ ]:
PROMPT_VERSION = "teacher_sensitivity_v1"

def save_teacher_artifacts(doc_text: str, teacher_out: SensitivityLabel) -> None:
    (ARTIFACTS_DIR / "inputs").mkdir(exist_ok=True)
    (ARTIFACTS_DIR / "teacher").mkdir(exist_ok=True)

    (ARTIFACTS_DIR / "inputs" / f"{DOC_HASH}.txt").write_text(doc_text, encoding="utf-8")

    meta = {
        "run_id": RUN_ID,
        "doc_hash": DOC_HASH,
        "pdf_path": str(PDF_PATH),
        "prompt_version": PROMPT_VERSION,
        "teacher_model": OPENAI_MODEL,
        "created_at": datetime.datetime.now().isoformat(),
    }
    (ARTIFACTS_DIR / "teacher" / f"{DOC_HASH}.meta.json").write_text(
        json.dumps(meta, indent=2, ensure_ascii=False), encoding="utf-8"
    )
    (ARTIFACTS_DIR / "teacher" / f"{DOC_HASH}.label.json").write_text(
        json.dumps(teacher_out.model_dump(), indent=2, ensure_ascii=False), encoding="utf-8"
    )

# Ejemplo:
# save_teacher_artifacts(raw_text, teacher_out)
print("OK. Cuando tengas teacher_out, llama save_teacher_artifacts(raw_text, teacher_out).")


## 7) Preparar dataset JSONL para entrenar un SLM local (student)
Esto genera un ejemplo SFT (messages + response).

🔴 Nota: Ollama no entrena por API. Este JSONL lo usarás con una tool de fine-tuning (LoRA) en local.


In [ ]:
STUDENT_SYSTEM = "Eres un clasificador binario de sensibilidad documental. Responde SOLO JSON válido."
STUDENT_USER_TEMPLATE = """Clasifica sensibilidad (0=no sensible,1=sensible) según:
- 1 si hay IBAN, NIF/CIF, datos bancarios, datos de pago, identificadores fiscales o info financiera identificable.
- 0 si no.
Devuelve SOLO JSON con: label, confidence, signals, rationale_short.

Texto:
<<<
{doc_text}
>>>
"""

def build_sft_example(doc_text: str, teacher_out: SensitivityLabel) -> dict:
    return {
        "messages": [
            {"role": "system", "content": STUDENT_SYSTEM},
            {"role": "user", "content": STUDENT_USER_TEMPLATE.format(doc_text=doc_text)},
        ],
        "response": json.dumps(teacher_out.model_dump(), ensure_ascii=False),
        "meta": {
            "doc_hash": DOC_HASH,
            "teacher_model": OPENAI_MODEL,
            "prompt_version": PROMPT_VERSION,
        },
    }

def append_jsonl(path: Path, obj: dict) -> None:
    with path.open("a", encoding="utf-8") as f:
        f.write(json.dumps(obj, ensure_ascii=False) + "\n")

# Ejemplo:
# sft_path = ARTIFACTS_DIR / "student_train.jsonl"
# append_jsonl(sft_path, build_sft_example(raw_text, teacher_out))
print("OK. Cuando tengas teacher_out, construye el JSONL con build_sft_example + append_jsonl.")


## 8) Llamada a Ollama/Qwen (student) en modo JSON
Hacemos inferencia con Ollama `/api/chat` y forzamos `format: "json"`.


In [ ]:
def ollama_chat_json(model: str, system: str, user: str) -> dict:
    url = f"{OLLAMA_URL}/api/chat"
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": system},
            {"role": "user", "content": user},
        ],
        "stream": False,
        "format": "json",
    }
    r = requests.post(url, json=payload, timeout=120)
    r.raise_for_status()
    data = r.json()
    content = data.get("message", {}).get("content", "")
    try:
        return json.loads(content)
    except json.JSONDecodeError:
        return {"_raw": content, "_error": "not_json"}

# Ejemplo:
# student_pred = ollama_chat_json(
#     model=OLLAMA_MODEL,
#     system=STUDENT_SYSTEM,
#     user=STUDENT_USER_TEMPLATE.format(doc_text=raw_text[:4000])
# )
# print(student_pred)

print("Descomenta para ejecutar. Requiere Ollama corriendo y el modelo descargado.")


## 9) Parse/validación del output del student con Pydantic
Si falla, tu pipeline puede enviar a revisión humana o reintentar con prompt más estricto.


In [ ]:
def parse_student_output(student_json: dict) -> SensitivityLabel:
    return SensitivityLabel.from_any(student_json)

# Ejemplo:
# student_label = parse_student_output(student_pred)
# print(student_label.model_dump())

print("OK. Listo para validar cuando ejecutes el student.")


## 10) Knowledge Base Log (JSONL)
Registramos cada documento procesado con:
- doc_hash
- teacher label
- student label
- versiones y artifacts


In [ ]:
KB_EVENTS = KB_DIR / "events.jsonl"

def log_kb_event(
    doc_hash: str,
    pdf_path: str,
    teacher_out: SensitivityLabel,
    student_out: Optional[SensitivityLabel] = None,
    notes: Optional[str] = None,
) -> dict:
    event = {
        "ts": datetime.datetime.now().isoformat(),
        "doc_hash": doc_hash,
        "pdf_path": pdf_path,
        "teacher": teacher_out.model_dump(),
        "student": student_out.model_dump() if student_out else None,
        "versions": {
            "prompt_version": PROMPT_VERSION,
            "teacher_model": OPENAI_MODEL,
            "ollama_model": OLLAMA_MODEL,
        },
        "artifacts_dir": str(ARTIFACTS_DIR),
        "notes": notes,
    }
    append_jsonl(KB_EVENTS, event)
    return event

# Ejemplo:
# event = log_kb_event(DOC_HASH, str(PDF_PATH), teacher_out, student_label)
print("OK. Esto crea knowledge_base/events.jsonl")


## 11) (Opcional) Knowledge Base en SQLite
Si quieres consultas rápidas, indexación y reporting.


In [ ]:
import sqlite3

KB_DB = KB_DIR / "kb.sqlite"

def init_kb_db(db_path: Path):
    con = sqlite3.connect(str(db_path))
    cur = con.cursor()
    cur.execute("""
    CREATE TABLE IF NOT EXISTS events (
        ts TEXT,
        doc_hash TEXT,
        pdf_path TEXT,
        prompt_version TEXT,
        teacher_model TEXT,
        ollama_model TEXT,
        teacher_json TEXT,
        student_json TEXT,
        artifacts_dir TEXT,
        notes TEXT
    )
    """)
    cur.execute("CREATE INDEX IF NOT EXISTS idx_doc_hash ON events(doc_hash)")
    con.commit()
    con.close()

def insert_kb_db(db_path: Path, event: dict):
    con = sqlite3.connect(str(db_path))
    cur = con.cursor()
    cur.execute(
        """INSERT INTO events VALUES (?,?,?,?,?,?,?,?,?,?)""",
        (
            event["ts"],
            event["doc_hash"],
            event["pdf_path"],
            event["versions"]["prompt_version"],
            event["versions"]["teacher_model"],
            event["versions"]["ollama_model"],
            json.dumps(event["teacher"], ensure_ascii=False),
            json.dumps(event["student"], ensure_ascii=False) if event["student"] else None,
            event["artifacts_dir"],
            event.get("notes"),
        ),
    )
    con.commit()
    con.close()

init_kb_db(KB_DB)
print("KB DB ready:", KB_DB.resolve())


## 12) Ejecutar el flujo completo (pasos)
1) Extraer texto (ya lo hicimos)
2) Teacher:
   - `teacher_out = teacher_label_document(raw_text)`
   - `save_teacher_artifacts(raw_text, teacher_out)`
3) Dataset para entrenamiento:
   - `append_jsonl(ARTIFACTS_DIR/'student_train.jsonl', build_sft_example(raw_text, teacher_out))`
4) Student (Ollama):
   - `student_pred = ollama_chat_json(...)`
   - `student_label = parse_student_output(student_pred)`
5) KB log:
   - `event = log_kb_event(...)`
   - `insert_kb_db(KB_DB, event)`

🧠 Entrenamiento real del SLM:
- Con el JSONL generado, haces fine-tuning (LoRA) con una herramienta local.
- Luego creas un modelo en Ollama con un Modelfile y vuelves a evaluar.
